# 시간대별 혼잡도와 역세권 건물 상관관계 분석

이 노트북은 지하철 역의 시간대별 혼잡도와 역세권 건물 특성 간의 상관관계를 분석합니다.

## 분석 대상
- **2호선, 4호선, 5호선** 역으로 한정

## 분석 내용
1. 역세권 건물 특성 집계 (용도별 건물 수, 연면적, 세대수 등)
2. 시간대별 혼잡도 데이터 분석
3. 상관관계 분석 및 시각화
4. 건물 용도와 시간대별 혼잡도 패턴 분석

In [18]:
import sqlite3
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from scipy import stats
from IPython.display import display
import warnings
warnings.filterwarnings('ignore')

# 설정
DB_PATH = "../db/subway.db"

def get_connection():
    return sqlite3.connect(DB_PATH)

print("라이브러리 로드 완료")

라이브러리 로드 완료


## 1. 데이터 로드

In [ ]:
conn = get_connection()

# 분석 대상 호선 설정
TARGET_LINES = ['2호선', '4호선', '5호선']

# 역 정보 로드 (2, 4, 5호선만)
stations_query = """
SELECT 
    s.station_id,
    s.station_name_kr,
    sr.station_code,
    sr.admin_dong_code,
    sr.admin_dong_name,
    l.line_name
FROM Stations s
JOIN Station_Routes sr ON s.station_id = sr.station_id
JOIN Lines l ON sr.line_id = l.line_id
WHERE l.line_name IN ('2호선', '4호선', '5호선')
"""
df_stations = pd.read_sql(stations_query, conn)

# 대상 역 ID 목록
target_station_ids = df_stations['station_id'].unique().tolist()
target_station_codes = df_stations['station_code'].unique().tolist()

# 역세권 건물 정보 로드 (대상 역만)
buildings_query = f"""
SELECT 
    b.*,
    s.station_name_kr
FROM Station_Catchment_Buildings b
JOIN Stations s ON b.station_id = s.station_id
WHERE b.station_id IN ({','.join(map(str, target_station_ids))})
"""
df_buildings = pd.read_sql(buildings_query, conn)

# 혼잡도 데이터 로드 (대상 역만)
congestion_query = f"""
SELECT * FROM Station_Congestion
WHERE station_code IN ({','.join([f"'{c}'" for c in target_station_codes])})
"""
df_congestion = pd.read_sql(congestion_query, conn)

print(f"분석 대상 호선: {TARGET_LINES}")
print(f"역 정보: {len(df_stations):,} rows ({df_stations['station_id'].nunique()}개 역)")
print(f"역세권 건물: {len(df_buildings):,} rows")
print(f"혼잡도 데이터: {len(df_congestion):,} rows")

# 호선별 역 수
print(f"\n호선별 역 수:")
print(df_stations.groupby('line_name')['station_id'].nunique())

display(df_congestion.head())

### 1.1 혼잡도 데이터 기본 정보

In [20]:
print("=== 혼잡도 데이터 기본 정보 ===")
print(f"분기 코드: {df_congestion['quarter_code'].unique()}")
print(f"요일 구분: 0=평일, 1=토요일, 2=일요일")
print(f"시간대 범위: {df_congestion['time_slot'].min()} ~ {df_congestion['time_slot'].max()}")
print(f"혼잡도 범위: {df_congestion['congestion_level'].min():.1f} ~ {df_congestion['congestion_level'].max():.1f}")
print(f"\n역 코드 수: {df_congestion['station_code'].nunique()}")

=== 혼잡도 데이터 기본 정보 ===
분기 코드: ['20234' '20241' '20242' '20244' '20251']
요일 구분: 0=평일, 1=토요일, 2=일요일
시간대 범위: 0 ~ 38
혼잡도 범위: 0.0 ~ 192.8

역 코드 수: 276


In [21]:
# 시간대 슬롯을 실제 시간으로 변환 (05:30 = 0, 06:00 = 1, ...)
def slot_to_time(slot):
    """시간대 슬롯을 실제 시간 문자열로 변환"""
    base_hour = 5
    base_minute = 30
    total_minutes = base_hour * 60 + base_minute + slot * 30
    hour = (total_minutes // 60) % 24
    minute = total_minutes % 60
    return f"{hour:02d}:{minute:02d}"

def slot_to_hour(slot):
    """시간대 슬롯을 시간(hour)으로 변환"""
    base_hour = 5
    base_minute = 30
    total_minutes = base_hour * 60 + base_minute + slot * 30
    return (total_minutes // 60) % 24

# 시간대 그룹 분류
def categorize_time_period(slot):
    """시간대를 출근/점심/퇴근/야간 등으로 분류"""
    hour = slot_to_hour(slot)
    if 0 <= hour < 6:
        return '심야(00-06)'
    elif 6 <= hour < 9:
        return '출근(06-09)'
    elif 9 <= hour < 12:
        return '오전(09-12)'
    elif 12 <= hour < 14:
        return '점심(12-14)'
    elif 14 <= hour < 18:
        return '오후(14-18)'
    elif 18 <= hour < 21:
        return '퇴근(18-21)'
    else:
        return '야간(21-24)'

df_congestion['time_str'] = df_congestion['time_slot'].apply(slot_to_time)
df_congestion['hour'] = df_congestion['time_slot'].apply(slot_to_hour)
df_congestion['time_period'] = df_congestion['time_slot'].apply(categorize_time_period)

print("시간대 슬롯 매핑 예시:")
for slot in [0, 1, 2, 5, 10, 15, 20, 25, 30, 35, 38]:
    print(f"  slot {slot:2d} -> {slot_to_time(slot)} ({categorize_time_period(slot)})")

시간대 슬롯 매핑 예시:
  slot  0 -> 05:30 (심야(00-06))
  slot  1 -> 06:00 (출근(06-09))
  slot  2 -> 06:30 (출근(06-09))
  slot  5 -> 08:00 (출근(06-09))
  slot 10 -> 10:30 (오전(09-12))
  slot 15 -> 13:00 (점심(12-14))
  slot 20 -> 15:30 (오후(14-18))
  slot 25 -> 18:00 (퇴근(18-21))
  slot 30 -> 20:30 (퇴근(18-21))
  slot 35 -> 23:00 (야간(21-24))
  slot 38 -> 00:30 (심야(00-06))


## 2. 역별 건물 특성 집계

In [22]:
# 용도 카테고리 분류
def categorize_usage(usage_type):
    if pd.isna(usage_type):
        return '기타'
    usage_type = str(usage_type)
    if '주택' in usage_type or '공동주택' in usage_type:
        return '주거'
    elif '근린생활' in usage_type or '판매' in usage_type:
        return '상업'
    elif '업무' in usage_type:
        return '업무'
    elif '교육' in usage_type or '학교' in usage_type:
        return '교육'
    elif '숙박' in usage_type:
        return '숙박'
    elif '공장' in usage_type or '창고' in usage_type:
        return '산업'
    elif '종교' in usage_type or '문화' in usage_type:
        return '문화종교'
    else:
        return '기타'

df_buildings['usage_category'] = df_buildings['usage_type'].apply(categorize_usage)

# 역별 건물 특성 집계
station_building_stats = df_buildings.groupby('station_id').agg({
    'id': 'count',
    'height': 'mean',
    'floor_area': 'sum',
    'households': 'sum',
    'families': 'sum'
}).rename(columns={
    'id': 'building_count',
    'height': 'avg_height',
    'floor_area': 'total_floor_area',
    'households': 'total_households',
    'families': 'total_families'
}).reset_index()

# 용도별 건물 수 피벗
usage_pivot = df_buildings.groupby(['station_id', 'usage_category']).size().unstack(fill_value=0)
usage_pivot.columns = [f'cnt_{col}' for col in usage_pivot.columns]
usage_pivot = usage_pivot.reset_index()

# 합치기
station_building_stats = station_building_stats.merge(usage_pivot, on='station_id', how='left')

# 역명 추가
station_names = df_buildings[['station_id', 'station_name_kr']].drop_duplicates()
station_building_stats = station_building_stats.merge(station_names, on='station_id', how='left')

print(f"역별 건물 특성 집계: {len(station_building_stats)} 역")
display(station_building_stats.head())

역별 건물 특성 집계: 235 역


,station_id,building_count,avg_height,total_floor_area,total_households,total_families,cnt_교육,cnt_기타,cnt_문화종교,cnt_산업,cnt_상업,cnt_숙박,cnt_업무,cnt_주거,station_name_kr
0,1,2642,4.852903,170912.290040,6086.0,1138.0,10,906,20,30,724,20,106,826,서울
1,2,1588,7.841936,184587.381544,5820.0,1024.0,40,516,76,6,548,58,276,68,시청
2,3,1281,6.861591,129335.944245,3724.0,685.0,27,458,17,3,570,24,110,72,종각
3,4,6852,2.827830,357611.362872,13998.0,1818.0,36,1374,102,57,3729,219,111,1224,종로3가
4,5,2274,3.312860,134419.485557,5262.0,742.0,6,561,6,75,1236,27,60,303,종로5가


## 3. 역별 시간대별 혼잡도 집계

In [23]:
# station_code를 station_id로 매핑
station_code_to_id = df_stations[['station_id', 'station_code']].drop_duplicates()

# 혼잡도 데이터에 station_id 추가
df_congestion = df_congestion.merge(station_code_to_id, on='station_code', how='left')

print(f"혼잡도 데이터 중 station_id 매핑된 수: {df_congestion['station_id'].notna().sum():,}")

혼잡도 데이터 중 station_id 매핑된 수: 271,142


In [24]:
# 평일 데이터만 사용 (day_of_week = 0)
df_congestion_weekday = df_congestion[df_congestion['day_of_week'] == 0].copy()

# 역별, 시간대별 평균 혼잡도 (상행/하행 평균)
congestion_by_time = df_congestion_weekday.groupby(['station_id', 'time_period']).agg({
    'congestion_level': 'mean'
}).reset_index()

# 피벗
congestion_pivot = congestion_by_time.pivot(
    index='station_id', 
    columns='time_period', 
    values='congestion_level'
).reset_index()

# 컬럼명 정리
congestion_pivot.columns.name = None

# 역별 전체 평균 혼잡도도 계산
avg_congestion = df_congestion_weekday.groupby('station_id')['congestion_level'].mean().reset_index()
avg_congestion.columns = ['station_id', 'avg_congestion']

congestion_pivot = congestion_pivot.merge(avg_congestion, on='station_id', how='left')

print(f"혼잡도 피벗 데이터: {len(congestion_pivot)} 역")
display(congestion_pivot.head())

혼잡도 피벗 데이터: 241 역


,station_id,심야(00-06),야간(21-24),오전(09-12),오후(14-18),점심(12-14),출근(06-09),퇴근(18-21),avg_congestion
0,1,8.213333,22.913333,31.155833,38.745625,31.198750,32.714167,39.506667,31.208718
1,2,8.546667,31.303333,32.465833,43.425000,34.118750,37.245000,50.915833,36.438333
2,3,5.156667,23.215000,28.366667,37.986250,29.185000,27.788333,43.791667,30.130000
3,4,8.615556,26.205556,33.620000,45.277500,34.495833,40.775000,45.686111,35.994103
4,5,6.410000,21.771667,31.478333,38.655000,31.570000,33.333333,40.105000,31.150769


## 4. 데이터 병합 및 상관관계 분석

In [25]:
# 건물 특성과 혼잡도 데이터 병합
df_analysis = station_building_stats.merge(congestion_pivot, on='station_id', how='inner')

print(f"분석 대상 역 수: {len(df_analysis)}")
display(df_analysis.head())

분석 대상 역 수: 225


,station_id,building_count,avg_height,total_floor_area,total_households,total_families,cnt_교육,cnt_기타,cnt_문화종교,cnt_산업,...,cnt_주거,station_name_kr,심야(00-06),야간(21-24),오전(09-12),오후(14-18),점심(12-14),출근(06-09),퇴근(18-21),avg_congestion
0,1,2642,4.852903,170912.290040,6086.0,1138.0,10,906,20,30,...,826,서울,8.213333,22.913333,31.155833,38.745625,31.198750,32.714167,39.506667,31.208718
1,2,1588,7.841936,184587.381544,5820.0,1024.0,40,516,76,6,...,68,시청,8.546667,31.303333,32.465833,43.425000,34.118750,37.245000,50.915833,36.438333
2,3,1281,6.861591,129335.944245,3724.0,685.0,27,458,17,3,...,72,종각,5.156667,23.215000,28.366667,37.986250,29.185000,27.788333,43.791667,30.130000
3,4,6852,2.827830,357611.362872,13998.0,1818.0,36,1374,102,57,...,1224,종로3가,8.615556,26.205556,33.620000,45.277500,34.495833,40.775000,45.686111,35.994103
4,5,2274,3.312860,134419.485557,5262.0,742.0,6,561,6,75,...,303,종로5가,6.410000,21.771667,31.478333,38.655000,31.570000,33.333333,40.105000,31.150769


In [26]:
# 상관관계 분석을 위한 변수 선택
building_features = ['building_count', 'avg_height', 'total_floor_area', 'total_households']
usage_features = [c for c in df_analysis.columns if c.startswith('cnt_') and c != 'cnt_기타']
building_features.extend(usage_features)

time_features = ['심야(00-06)', '출근(06-09)', '오전(09-12)', '점심(12-14)', 
                 '오후(14-18)', '퇴근(18-21)', '야간(21-24)', 'avg_congestion']
time_features = [f for f in time_features if f in df_analysis.columns]

# 상관계수 계산
all_features = building_features + time_features
corr_matrix = df_analysis[all_features].corr()

# 건물 특성과 혼잡도 간의 상관관계만 추출
corr_building_congestion = corr_matrix.loc[building_features, time_features]

print("건물 특성과 시간대별 혼잡도 간 상관계수:")
display(corr_building_congestion.round(3))

건물 특성과 시간대별 혼잡도 간 상관계수:


,심야(00-06),출근(06-09),오전(09-12),점심(12-14),오후(14-18),퇴근(18-21),야간(21-24),avg_congestion
building_count,0.142,0.132,0.161,0.181,0.140,0.120,0.148,0.148
avg_height,0.031,0.063,0.094,0.112,0.158,0.162,0.180,0.133
total_floor_area,0.302,0.258,0.258,0.242,0.223,0.225,0.274,0.257
total_households,0.221,0.208,0.220,0.225,0.194,0.185,0.229,0.216
cnt_교육,0.035,0.023,0.108,0.171,0.164,0.120,0.141,0.120
cnt_문화종교,0.012,0.061,0.181,0.255,0.220,0.162,0.137,0.165
cnt_산업,-0.071,-0.042,0.028,0.108,0.082,0.067,0.059,0.044
cnt_상업,0.030,0.068,0.167,0.241,0.203,0.160,0.174,0.165
cnt_숙박,0.030,0.065,0.156,0.233,0.197,0.143,0.156,0.155
cnt_업무,0.073,0.100,0.242,0.343,0.341,0.300,0.315,0.274


In [27]:
# 상관관계 히트맵
fig = go.Figure(data=go.Heatmap(
    z=corr_building_congestion.values,
    x=corr_building_congestion.columns,
    y=corr_building_congestion.index,
    colorscale='RdBu_r',
    zmid=0,
    text=corr_building_congestion.values.round(2),
    texttemplate='%{text}',
    textfont={'size': 9},
    hoverongaps=False,
    colorbar=dict(title='상관계수')
))

fig.update_layout(
    title='역세권 건물 특성과 시간대별 혼잡도 상관관계',
    xaxis_title='시간대별 혼잡도',
    yaxis_title='건물 특성',
    height=500,
    width=800
)

fig.show()

## 5. 주요 상관관계 상세 분석

In [28]:
# 주거 건물과 시간대별 혼잡도 관계
if 'cnt_주거' in df_analysis.columns:
    time_cols = [c for c in time_features if c != 'avg_congestion' and c in df_analysis.columns]
    n_cols = len(time_cols)
    
    fig = make_subplots(rows=2, cols=4, subplot_titles=time_cols[:8])
    
    positions = [(1,1), (1,2), (1,3), (1,4), (2,1), (2,2), (2,3), (2,4)]

    for i, col in enumerate(time_cols[:8]):
        if col in df_analysis.columns:
            corr, _ = stats.pearsonr(
                df_analysis['cnt_주거'].fillna(0), 
                df_analysis[col].fillna(0)
            )
            
            pos = positions[i]
            fig.add_trace(
                go.Scatter(
                    x=df_analysis['cnt_주거'],
                    y=df_analysis[col],
                    mode='markers',
                    marker=dict(size=5, opacity=0.6),
                    text=df_analysis['station_name_kr'],
                    name=f'r={corr:.2f}',
                    showlegend=False
                ),
                row=pos[0], col=pos[1]
            )
            
            # 추세선 추가
            x_valid = df_analysis['cnt_주거'].fillna(0)
            y_valid = df_analysis[col].fillna(0)
            z = np.polyfit(x_valid, y_valid, 1)
            p = np.poly1d(z)
            x_line = np.linspace(x_valid.min(), x_valid.max(), 100)
            fig.add_trace(
                go.Scatter(x=x_line, y=p(x_line), mode='lines', 
                          line=dict(color='red', dash='dash'),
                          showlegend=False),
                row=pos[0], col=pos[1]
            )

    fig.update_layout(
        height=500, 
        width=1000,
        title_text='주거 건물 수와 시간대별 혼잡도 관계'
    )
    fig.update_xaxes(title_text='주거 건물 수')
    fig.update_yaxes(title_text='혼잡도')
    fig.show()

In [29]:
# 상업 건물과 시간대별 혼잡도 관계
if 'cnt_상업' in df_analysis.columns:
    time_cols = [c for c in time_features if c != 'avg_congestion' and c in df_analysis.columns]
    
    fig = make_subplots(rows=2, cols=4, subplot_titles=time_cols[:8])
    positions = [(1,1), (1,2), (1,3), (1,4), (2,1), (2,2), (2,3), (2,4)]

    for i, col in enumerate(time_cols[:8]):
        if col in df_analysis.columns:
            corr, _ = stats.pearsonr(
                df_analysis['cnt_상업'].fillna(0), 
                df_analysis[col].fillna(0)
            )
            
            pos = positions[i]
            fig.add_trace(
                go.Scatter(
                    x=df_analysis['cnt_상업'],
                    y=df_analysis[col],
                    mode='markers',
                    marker=dict(size=5, opacity=0.6, color='orange'),
                    text=df_analysis['station_name_kr'],
                    showlegend=False
                ),
                row=pos[0], col=pos[1]
            )

    fig.update_layout(
        height=500, 
        width=1000,
        title_text='상업 건물 수와 시간대별 혼잡도 관계'
    )
    fig.update_xaxes(title_text='상업 건물 수')
    fig.update_yaxes(title_text='혼잡도')
    fig.show()

In [30]:
# 업무 건물과 시간대별 혼잡도 관계
if 'cnt_업무' in df_analysis.columns:
    time_cols = [c for c in time_features if c != 'avg_congestion' and c in df_analysis.columns]
    
    fig = make_subplots(rows=2, cols=4, subplot_titles=time_cols[:8])
    positions = [(1,1), (1,2), (1,3), (1,4), (2,1), (2,2), (2,3), (2,4)]

    for i, col in enumerate(time_cols[:8]):
        if col in df_analysis.columns:
            corr, _ = stats.pearsonr(
                df_analysis['cnt_업무'].fillna(0), 
                df_analysis[col].fillna(0)
            )
            
            pos = positions[i]
            fig.add_trace(
                go.Scatter(
                    x=df_analysis['cnt_업무'],
                    y=df_analysis[col],
                    mode='markers',
                    marker=dict(size=5, opacity=0.6, color='green'),
                    text=df_analysis['station_name_kr'],
                    showlegend=False
                ),
                row=pos[0], col=pos[1]
            )

    fig.update_layout(
        height=500, 
        width=1000,
        title_text='업무 건물 수와 시간대별 혼잡도 관계'
    )
    fig.update_xaxes(title_text='업무 건물 수')
    fig.update_yaxes(title_text='혼잡도')
    fig.show()

## 6. 시간대별 혼잡도 패턴 분석

In [31]:
# 전체 역의 시간대별 평균 혼잡도 패턴
time_pattern = df_congestion_weekday.groupby('time_slot')['congestion_level'].mean().reset_index()
time_pattern['time_str'] = time_pattern['time_slot'].apply(slot_to_time)

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=time_pattern['time_str'],
    y=time_pattern['congestion_level'],
    mode='lines+markers',
    line=dict(color='blue', width=2),
    marker=dict(size=6)
))

fig.update_layout(
    title='평일 시간대별 평균 혼잡도',
    xaxis_title='시간',
    yaxis_title='평균 혼잡도',
    height=400,
    xaxis=dict(tickangle=45)
)
fig.show()

In [32]:
# 건물 용도 비율 계산
usage_cols = [c for c in df_analysis.columns if c.startswith('cnt_')]
for col in usage_cols:
    ratio_col = col.replace('cnt_', 'ratio_')
    df_analysis[ratio_col] = df_analysis[col] / df_analysis['building_count']

# 주거 비율 상위/하위 역 비교
df_analysis['주거비율_그룹'] = pd.qcut(df_analysis['ratio_주거'].fillna(0), q=3, labels=['하위', '중위', '상위'])

# 그룹별 시간대 혼잡도 패턴
group_patterns = df_analysis.groupby('주거비율_그룹')[time_features].mean()

fig = go.Figure()
colors = {'하위': 'red', '중위': 'gray', '상위': 'blue'}

for group in ['하위', '중위', '상위']:
    if group in group_patterns.index:
        values = group_patterns.loc[group, [c for c in time_features if c != 'avg_congestion']]
        fig.add_trace(go.Bar(
            name=f'주거 비율 {group}',
            x=values.index,
            y=values.values,
            marker_color=colors[group]
        ))

fig.update_layout(
    title='주거 건물 비율별 시간대 혼잡도 비교',
    xaxis_title='시간대',
    yaxis_title='평균 혼잡도',
    barmode='group',
    height=400
)
fig.show()

In [33]:
# 업무 비율 상위/하위 역 비교
if 'ratio_업무' in df_analysis.columns:
    df_analysis['업무비율_그룹'] = pd.qcut(
        df_analysis['ratio_업무'].fillna(0).clip(lower=0.001), 
        q=3, 
        labels=['하위', '중위', '상위'],
        duplicates='drop'
    )

    group_patterns = df_analysis.groupby('업무비율_그룹')[time_features].mean()

    fig = go.Figure()
    colors = {'하위': 'lightgreen', '중위': 'gray', '상위': 'darkgreen'}

    for group in ['하위', '중위', '상위']:
        if group in group_patterns.index:
            values = group_patterns.loc[group, [c for c in time_features if c != 'avg_congestion']]
            fig.add_trace(go.Bar(
                name=f'업무 비율 {group}',
                x=values.index,
                y=values.values,
                marker_color=colors[group]
            ))

    fig.update_layout(
        title='업무 건물 비율별 시간대 혼잡도 비교',
        xaxis_title='시간대',
        yaxis_title='평균 혼잡도',
        barmode='group',
        height=400
    )
    fig.show()

## 7. 역 클러스터링 (건물 특성 기반)

In [34]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# 클러스터링에 사용할 피처
cluster_features = ['building_count', 'avg_height', 'total_floor_area']
ratio_features = [c for c in df_analysis.columns if c.startswith('ratio_') and c not in ['ratio_기타']]
ratio_features = [f for f in ratio_features if f in df_analysis.columns and not f.endswith('_그룹')]

cluster_features.extend(ratio_features[:5])  # 상위 5개 용도만 사용
cluster_features = [f for f in cluster_features if f in df_analysis.columns]

print(f"클러스터링 피처: {cluster_features}")

# 결측치 처리 및 스케일링
X = df_analysis[cluster_features].fillna(0)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# K-means 클러스터링
kmeans = KMeans(n_clusters=4, random_state=42, n_init=10)
df_analysis['cluster'] = kmeans.fit_predict(X_scaled)

print("\n클러스터별 역 수:")
print(df_analysis['cluster'].value_counts().sort_index())

클러스터링 피처: ['building_count', 'avg_height', 'total_floor_area', 'ratio_교육', 'ratio_문화종교', 'ratio_산업', 'ratio_상업', 'ratio_숙박']

클러스터별 역 수:
cluster
0      5
1     51
2    120
3     49
Name: count, dtype: int64


In [35]:
# 클러스터별 시간대 혼잡도 패턴
time_cols_for_viz = [c for c in time_features if c != 'avg_congestion' and c in df_analysis.columns]
cluster_patterns = df_analysis.groupby('cluster')[time_cols_for_viz].mean()

fig = go.Figure()
colors = ['blue', 'red', 'green', 'purple']

for cluster in sorted(cluster_patterns.index):
    fig.add_trace(go.Scatter(
        x=time_cols_for_viz,
        y=cluster_patterns.loc[cluster].values,
        mode='lines+markers',
        name=f'클러스터 {cluster}',
        line=dict(color=colors[cluster % len(colors)], width=2)
    ))

fig.update_layout(
    title='클러스터별 시간대 혼잡도 패턴',
    xaxis_title='시간대',
    yaxis_title='평균 혼잡도',
    height=450
)
fig.show()

In [36]:
# 클러스터별 건물 특성 요약
summary_cols = ['building_count', 'avg_height', 'total_floor_area', 'total_households', 'avg_congestion']
usage_cols_summary = [c for c in df_analysis.columns if c.startswith('cnt_') and c != 'cnt_기타']
summary_cols.extend(usage_cols_summary)
summary_cols = [c for c in summary_cols if c in df_analysis.columns]

cluster_summary = df_analysis.groupby('cluster')[summary_cols].mean().round(1)

print("클러스터별 특성 평균:")
display(cluster_summary.T)

클러스터별 특성 평균:


cluster,0,1,2,3
building_count,1139.6,3209.9,1664.2,649.5
avg_height,7.1,7.5,5.8,9.5
total_floor_area,73968.9,289360.4,128166.2,54026.0
total_households,3013.2,8927.3,4236.8,2378.0
avg_congestion,35.0,37.4,31.4,31.6
cnt_교육,8.4,23.9,12.5,19.9
cnt_문화종교,1.8,14.7,6.6,9.6
cnt_산업,114.8,26.9,3.9,1.6
cnt_상업,264.2,998.8,274.2,155.3
cnt_숙박,1.6,34.3,2.6,3.7


In [37]:
# 각 클러스터 대표 역
print("클러스터별 대표 역:")
for cluster in sorted(df_analysis['cluster'].unique()):
    cluster_data = df_analysis[df_analysis['cluster'] == cluster]
    stations = cluster_data.nlargest(5, 'avg_congestion')['station_name_kr'].tolist()
    avg_cong = cluster_data['avg_congestion'].mean()
    print(f"\n클러스터 {cluster} (평균 혼잡도: {avg_cong:.1f}):")
    print(f"  {', '.join(stations)}")

클러스터별 대표 역:

클러스터 0 (평균 혼잡도: 35.0):
  가산디지털단지, 신도림, 뚝섬, 성수, 도림천

클러스터 1 (평균 혼잡도: 37.4):
  석촌, 길동, 역삼, 교대, 서울대입구

클러스터 2 (평균 혼잡도: 31.4):
  송파, 방배, 서초, 낙성대, 남태령

클러스터 3 (평균 혼잡도: 31.6):
  강남, 어린이대공원, 혜화, 압구정, 안국


## 8. 출퇴근 시간대 혼잡도 차이 분석

In [38]:
# 출근 vs 퇴근 혼잡도 차이
if '출근(06-09)' in df_analysis.columns and '퇴근(18-21)' in df_analysis.columns:
    df_analysis['출퇴근_차이'] = df_analysis['퇴근(18-21)'] - df_analysis['출근(06-09)']
    
    # 출근 혼잡도가 높은 역 (주거 지역 추정)
    morning_heavy = df_analysis.nlargest(10, '출근(06-09)')[['station_name_kr', '출근(06-09)', '퇴근(18-21)', 'cnt_주거', 'cnt_업무']]
    
    # 퇴근 혼잡도가 높은 역 (업무 지역 추정)
    evening_heavy = df_analysis.nlargest(10, '퇴근(18-21)')[['station_name_kr', '출근(06-09)', '퇴근(18-21)', 'cnt_주거', 'cnt_업무']]
    
    print("=== 출근 시간대 혼잡도 Top 10 (주거 지역 추정) ===")
    display(morning_heavy)
    
    print("\n=== 퇴근 시간대 혼잡도 Top 10 (업무 지역 추정) ===")
    display(evening_heavy)

=== 출근 시간대 혼잡도 Top 10 (주거 지역 추정) ===


,station_name_kr,출근(06-09),퇴근(18-21),cnt_주거,cnt_업무
221,송파,76.753333,71.630000,472,8
220,석촌,71.905000,73.683333,2814,62
195,중곡,69.240000,59.251667,1942,5
196,어린이대공원,68.350000,63.440000,795,16
194,용마산,66.861667,57.560000,1722,3
193,사가정,65.771667,53.565000,2572,3
137,군자,64.474167,56.889167,4032,64
33,방배,63.790000,67.921667,561,35
138,아차산,63.663333,56.481667,1713,12
109,남태령,62.971667,62.635000,182,0



=== 퇴근 시간대 혼잡도 Top 10 (업무 지역 추정) ===


,station_name_kr,출근(06-09),퇴근(18-21),cnt_주거,cnt_업무
220,석촌,71.905000,73.683333,2814,62
221,송파,76.753333,71.630000,472,8
30,강남,51.328333,68.450000,110,118
32,서초,61.378333,68.243333,199,125
29,역삼,46.266667,67.933333,433,148
33,방배,63.790000,67.921667,561,35
222,문정,58.360000,66.085000,463,16
142,길동,58.531667,65.015000,933,37
196,어린이대공원,68.350000,63.440000,795,16
109,남태령,62.971667,62.635000,182,0


In [39]:
# 출근 vs 퇴근 혼잡도 산점도
if '출근(06-09)' in df_analysis.columns and '퇴근(18-21)' in df_analysis.columns:
    fig = go.Figure()
    
    # 주거 비율에 따른 색상
    fig.add_trace(go.Scatter(
        x=df_analysis['출근(06-09)'],
        y=df_analysis['퇴근(18-21)'],
        mode='markers',
        marker=dict(
            size=8,
            color=df_analysis['ratio_주거'].fillna(0),
            colorscale='RdYlBu_r',
            showscale=True,
            colorbar=dict(title='주거비율')
        ),
        text=df_analysis['station_name_kr'],
        hovertemplate='%{text}<br>출근: %{x:.1f}<br>퇴근: %{y:.1f}<extra></extra>'
    ))
    
    # 대각선 (출근=퇴근)
    max_val = max(df_analysis['출근(06-09)'].max(), df_analysis['퇴근(18-21)'].max())
    fig.add_trace(go.Scatter(
        x=[0, max_val],
        y=[0, max_val],
        mode='lines',
        line=dict(color='gray', dash='dash'),
        showlegend=False
    ))
    
    fig.update_layout(
        title='출근 vs 퇴근 시간대 혼잡도 (색상: 주거 비율)',
        xaxis_title='출근 시간대(06-09) 혼잡도',
        yaxis_title='퇴근 시간대(18-21) 혼잡도',
        height=500,
        width=600
    )
    fig.show()
    
    print("\n* 대각선 위: 퇴근 혼잡도 > 출근 혼잡도 (업무/상업 지역)")
    print("* 대각선 아래: 출근 혼잡도 > 퇴근 혼잡도 (주거 지역)")


* 대각선 위: 퇴근 혼잡도 > 출근 혼잡도 (업무/상업 지역)
* 대각선 아래: 출근 혼잡도 > 퇴근 혼잡도 (주거 지역)


## 9. 분석 결과 요약

In [40]:
print("=" * 70)
print("시간대별 혼잡도와 역세권 건물 상관관계 분석 결과 요약")
print("=" * 70)

print(f"\n[분석 개요]")
print(f"  - 분석 대상 역: {len(df_analysis)}개")
print(f"  - 총 건물 수: {df_buildings['id'].count():,}개")
print(f"  - 혼잡도 데이터: {len(df_congestion):,}건")

print(f"\n[주요 상관관계 (|r| > 0.1)]")
for building_col in building_features:
    if building_col in df_analysis.columns:
        significant_corrs = []
        for time_col in time_features:
            if time_col in df_analysis.columns:
                corr = df_analysis[building_col].corr(df_analysis[time_col])
                if abs(corr) > 0.1:
                    significant_corrs.append((time_col, corr))
        if significant_corrs:
            print(f"  {building_col}:")
            for time_col, corr in sorted(significant_corrs, key=lambda x: abs(x[1]), reverse=True)[:3]:
                direction = "양" if corr > 0 else "음"
                print(f"    - {time_col}: r={corr:.3f} ({direction}의 상관)")

print(f"\n[클러스터 특성]")
for cluster in sorted(df_analysis['cluster'].unique()):
    cluster_data = df_analysis[df_analysis['cluster'] == cluster]
    n_stations = len(cluster_data)
    avg_cong = cluster_data['avg_congestion'].mean()
    
    # 주요 건물 용도
    usage_cols = [c for c in cluster_data.columns if c.startswith('cnt_') and c != 'cnt_기타']
    if usage_cols:
        main_usage = max(usage_cols, key=lambda x: cluster_data[x].mean())
        main_usage_name = main_usage.replace('cnt_', '')
    else:
        main_usage_name = 'N/A'
    
    print(f"  클러스터 {cluster}: {n_stations}개 역, 평균 혼잡도 {avg_cong:.1f}, 주요 용도: {main_usage_name}")

conn.close()
print("\n" + "=" * 70)
print("분석 완료!")

시간대별 혼잡도와 역세권 건물 상관관계 분석 결과 요약

[분석 개요]
  - 분석 대상 역: 225개
  - 총 건물 수: 412,405개
  - 혼잡도 데이터: 271,142건

[주요 상관관계 (|r| > 0.1)]
  building_count:
    - 점심(12-14): r=0.181 (양의 상관)
    - 오전(09-12): r=0.161 (양의 상관)
    - avg_congestion: r=0.148 (양의 상관)
  avg_height:
    - 야간(21-24): r=0.180 (양의 상관)
    - 퇴근(18-21): r=0.162 (양의 상관)
    - 오후(14-18): r=0.158 (양의 상관)
  total_floor_area:
    - 심야(00-06): r=0.302 (양의 상관)
    - 야간(21-24): r=0.274 (양의 상관)
    - 오전(09-12): r=0.258 (양의 상관)
  total_households:
    - 야간(21-24): r=0.229 (양의 상관)
    - 점심(12-14): r=0.225 (양의 상관)
    - 심야(00-06): r=0.221 (양의 상관)
  cnt_교육:
    - 점심(12-14): r=0.171 (양의 상관)
    - 오후(14-18): r=0.164 (양의 상관)
    - 야간(21-24): r=0.141 (양의 상관)
  cnt_문화종교:
    - 점심(12-14): r=0.255 (양의 상관)
    - 오후(14-18): r=0.220 (양의 상관)
    - 오전(09-12): r=0.181 (양의 상관)
  cnt_산업:
    - 점심(12-14): r=0.108 (양의 상관)
  cnt_상업:
    - 점심(12-14): r=0.241 (양의 상관)
    - 오후(14-18): r=0.203 (양의 상관)
    - 야간(21-24): r=0.174 (양의 상관)
  cnt_숙박:
    - 점심(12-14): r=0.